In [805]:
import random
import pandas as pd
import numpy as np
from dataclasses import dataclass, field
from itertools import product

In [820]:
DIM = 10
LST_LAYER = ["Reticle", "Prev1Reticle", "Prev2Reticle"]
LST_LAYER_NAME = ["L0", "L1", "L2"]
LST_SCALE = [1, 1, 150, 150, 150, 150, 15, 15, 15, 15]
LST_FEAT = ["Tx", "Ty", "Rot", "Nor", "Ex", "Ey", "RRot", "RMag", "ARRot", "ARMag"]
CHUCK_TYPE = 2

In [821]:
if CHUCK_TYPE == 0:
    LST_CHUCK = [["S1", "S2"]]
elif CHUCK_TYPE == 1:
    LST_CHUCK = [["BASE"]]
elif CHUCK_TYPE == 2:
    LST_CHUCK = [["S1", "S2"], ["BASE"]]

rng_part_name = np.random.default_rng()
rng_bias = np.random.default_rng()
rng_var = np.random.default_rng()

In [822]:
FACTOR_BIAS = {
    "Part": 35,
    "Reticle": 0.1,
    "Prev1Reticle": 0.05,
    "Prev2Reticle": 0.05,
    "Tool": 8,
    "Prev1Tool": 2,
    "Prev2Tool": 2,
    "ChuckID": 1,
}

In [823]:
def gen_part_name():
    _prefix = "TM"
    get_chr = lambda: chr(rng_part_name.integers(65,91))
    _suffix = f'{random.randint(0,99):02d}'
    return _prefix + get_chr() + get_chr() + _suffix

def gen_ret_names(part_name, layer_name, num):
    return [f'{part_name[2:6]}{layer_name}-{i}' for i in range(1, num+1)]

def gen_bias(feat, base=0):
    bias = rng_bias.random(size=[DIM,1]) - 0.5
    scale = 1 / np.array(LST_SCALE[:DIM]).reshape([DIM, -1])
    factor = FACTOR_BIAS[feat]
    return bias * scale * factor + base

In [824]:
@dataclass
class Part:
    id: int
    n_ret: list

    name: str=field(init=False)
    ret_name: dict=field(init=False)
    
    bias: list=field(init=False)
    ret_bias: dict=field(init=False)
    
    chuck_type: list=field(init=False)

    def __post_init__(self):
        self.name = gen_part_name()
        self.bias = gen_bias(feat="Part")
        self.ret_name = dict()
        self.ret_bias = dict()

        for lyr, lyr_name, num  in zip(LST_LAYER, LST_LAYER_NAME, self.n_ret):
            ret_names = gen_ret_names(self.name, lyr_name, num)
            self.ret_name[lyr] = ret_names

            for ret_name in ret_names:
                self.ret_bias[ret_name] = gen_bias(feat=lyr, base=self.bias)

        self.chuck_type = random.choice(LST_CHUCK )
        
    def __repr__(self):
        name = f'{"Part":<15}: {self.name}'
        reticle = "\n".join([f'{lyr:<15}: {self.ret_name[lyr]}' for lyr in LST_LAYER])
        chuck = f'{"Chuck Type":<15}: {self.chuck_type}'
        
        ret_bias = f'{"Reticle Bias":<15}:'
        for ret, bias in self.ret_bias.items():
            for idx, value in enumerate(bias):
                ret_bias += "\n"
                if idx == 0:
                    ret_bias += f'{ret:<9}: ['
                else:
                    ret_bias += f'{"":<12}'
                ret_bias += f'[{float(value):10.6f}]'
            ret_bias += "]"
        
        return name + "\n" + reticle + "\n" + chuck + "\n" + ret_bias


@dataclass
class Tool:
    id: int
    
    name: str=field(init=False)
    bias0: list=field(init=False)
    bias1: list=field(init=False)
    bias2: list=field(init=False)
    
    chuck0: list=field(init=False)
    chuck1: list=field(init=False)
    chuck2: list=field(init=False)
    
    def __post_init__(self):
        self.name = f'Tool-{self.id}'
        self.bias0 = gen_bias(feat="Tool")
        self.bias1 = gen_bias(feat="Prev1Tool")
        self.bias2 = gen_bias(feat="Prev2Tool")
        
        self.chuck0 = gen_bias(feat="Tool")
        self.chuck1 = gen_bias(feat="Prev1Tool")
        self.chuck2 = gen_bias(feat="Prev2Tool")

In [840]:
pp = Part(1, [2,1,2])
pp

Part           : TMRY35
Reticle        : ['RY35L0-1', 'RY35L0-2']
Prev1Reticle   : ['RY35L1-1']
Prev2Reticle   : ['RY35L2-1', 'RY35L2-2']
Chuck Type     : ['BASE']
Reticle Bias   :
RY35L0-1 : [[-15.437758]
            [ -0.886736]
            [ -0.064996]
            [  0.070332]
            [  0.060125]
            [  0.002260]
            [  0.633635]
            [ -0.861451]
            [  0.004570]
            [ -0.080480]]
RY35L0-2 : [[-15.425815]
            [ -0.855509]
            [ -0.065305]
            [  0.070546]
            [  0.060052]
            [  0.002328]
            [  0.634479]
            [ -0.859805]
            [  0.004618]
            [ -0.083530]]
RY35L1-1 : [[-15.458453]
            [ -0.896667]
            [ -0.065352]
            [  0.070323]
            [  0.060002]
            [  0.002444]
            [  0.634696]
            [ -0.858020]
            [  0.002502]
            [ -0.080768]]
RY35L2-1 : [[-15.474219]
            [ -0.878413]
            [ -0

In [812]:
tt = Tool(1)
tt

Tool(id=1, name='Tool-1', bias0=array([[-8.84983570e-01],
       [ 1.66307385e+00],
       [ 1.10968910e-03],
       [-2.02327969e-02],
       [-2.37878369e-02],
       [-1.13306033e-02],
       [ 1.77354255e-01],
       [ 1.55113388e-01],
       [-1.70416679e-01],
       [-2.50194976e-01]]), bias1=array([[ 0.10112421],
       [ 0.02784014],
       [-0.00399772],
       [ 0.00354327],
       [ 0.00011704],
       [-0.00398437],
       [ 0.03957658],
       [-0.02752728],
       [-0.01351166],
       [ 0.01836622]]), bias2=array([[ 0.6909677 ],
       [ 0.47842362],
       [-0.00298393],
       [ 0.00293014],
       [-0.00441402],
       [ 0.00274361],
       [-0.01904421],
       [-0.02204224],
       [-0.03691462],
       [-0.00292563]]), chuck0=array([[-1.87354766],
       [ 1.37022317],
       [-0.01416702],
       [ 0.00554359],
       [-0.01835493],
       [ 0.0074011 ],
       [-0.09861574],
       [-0.24351458],
       [ 0.19188577],
       [ 0.12224257]]), chuck1=array([[ 0.245

In [813]:
def gen_data(n_part, n_ret, n_tool):
    _part_map = [Part(i, n_ret) for i in range(n_part)]
    _tool_map = [Tool(i) for i in range(n_tool)]
    
    part_map = {part.name: part for part in _part_map}
    tool_map = {tool.name: tool for tool in _tool_map}
    return part_map, tool_map

n_part = 5
n_ret = [2,1,1]
n_tool = 10

part_map, tool_map = gen_data(n_part, n_ret, n_tool)

In [815]:
LST_FEAT = ["Part", "Reticle", "Prev1Reticle", "Tool", "Prev1Tool", "Prev2Tool", "ChuckID"]

lst_comb = []
for part_name, part in part_map.items():
    lst_tool = list(tool_map.keys())
    lst_feats = [[part_name], part.ret_name["Reticle"], part.ret_name["Prev1Reticle"], lst_tool[:5], lst_tool[5:], lst_tool[5:], part.chuck_type]
    lst_comb.extend(list(product(*lst_feats)))

df_data = pd.DataFrame(lst_comb, columns=LST_FEAT)
df_data


,Part,Reticle,Prev1Reticle,Tool,Prev1Tool,Prev2Tool,ChuckID
0,TMZD37,ZD37L0-1,ZD37L1-1,Tool-0,Tool-5,Tool-5,BASE
1,TMZD37,ZD37L0-1,ZD37L1-1,Tool-0,Tool-5,Tool-6,BASE
2,TMZD37,ZD37L0-1,ZD37L1-1,Tool-0,Tool-5,Tool-7,BASE
3,TMZD37,ZD37L0-1,ZD37L1-1,Tool-0,Tool-5,Tool-8,BASE
4,TMZD37,ZD37L0-1,ZD37L1-1,Tool-0,Tool-5,Tool-9,BASE
...,...,...,...,...,...,...,...
1245,TMBC88,BC88L0-2,BC88L1-1,Tool-4,Tool-9,Tool-5,BASE
1246,TMBC88,BC88L0-2,BC88L1-1,Tool-4,Tool-9,Tool-6,BASE
1247,TMBC88,BC88L0-2,BC88L1-1,Tool-4,Tool-9,Tool-7,BASE
1248,TMBC88,BC88L0-2,BC88L1-1,Tool-4,Tool-9,Tool-8,BASE


In [817]:
def cal_rn(row):
    _Part = row["Part"]
    _Ret0 = row["Reticle"]
    _Ret1 = row["Prev1Reticle"]
    _Tool0 = row["Tool"]
    _Tool1 = row["Prev1Tool"]
    _Tool2 = row["Prev2Tool"]
    _Chuck = row["ChuckID"]
    
    rn0 = part_map[_Part].ret_bias[_Ret0][0] + tool_map[_Tool0].bias0[0]
    rn1 = part_map[_Part].ret_bias[_Ret1][0] + tool_map[_Tool1].bias1[0]
    rn2 = tool_map[_Tool2].bias2[0] + 0
    
    if _Chuck != "BASE":
        rn0 += tool_map[_Tool0].chuck0[0]
        rn1 += tool_map[_Tool1].chuck1[0]
        rn2 += tool_map[_Tool2].chuck2[0]
        
    return float(rn0+rn1+rn2)
    
    
df_data["Tx"] = df_data.apply(cal_rn, axis=1)
df_data

,Part,Reticle,Prev1Reticle,Tool,Prev1Tool,Prev2Tool,ChuckID,Tx
0,TMZD37,ZD37L0-1,ZD37L1-1,Tool-0,Tool-5,Tool-5,BASE,-17.326031
1,TMZD37,ZD37L0-1,ZD37L1-1,Tool-0,Tool-5,Tool-6,BASE,-17.361442
2,TMZD37,ZD37L0-1,ZD37L1-1,Tool-0,Tool-5,Tool-7,BASE,-18.108073
3,TMZD37,ZD37L0-1,ZD37L1-1,Tool-0,Tool-5,Tool-8,BASE,-16.442876
4,TMZD37,ZD37L0-1,ZD37L1-1,Tool-0,Tool-5,Tool-9,BASE,-16.769005
...,...,...,...,...,...,...,...,...
1245,TMBC88,BC88L0-2,BC88L1-1,Tool-4,Tool-9,Tool-5,BASE,-21.929025
1246,TMBC88,BC88L0-2,BC88L1-1,Tool-4,Tool-9,Tool-6,BASE,-21.964435
1247,TMBC88,BC88L0-2,BC88L1-1,Tool-4,Tool-9,Tool-7,BASE,-22.711067
1248,TMBC88,BC88L0-2,BC88L1-1,Tool-4,Tool-9,Tool-8,BASE,-21.045870


In [ ]:
tool_map['Tool-4']